In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from transformers import ViTFeatureExtractor, ViTModel, ViTConfig
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

2025-04-29 14:25:28.696997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745954728.983820 1271041 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745954729.077656 1271041 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745954729.840763 1271041 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745954729.840792 1271041 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745954729.840794 1271041 computation_placer.cc:177] computation placer alr

In [2]:
data_path = Path('../MIMIC/physionet.org/content/mimic-cxr-jpg/get-zip/2.1.0/mimic-cxr-jpg-chest-radiographs-with-structured-labels-2.1.0')
icu_data_path = Path('../MIMIC/MIMICIV/physionet.org/files/mimiciv/3.1/icu')
icustay_path = icu_data_path / 'icustays.csv.gz'
hosp_data_path = Path('../MIMIC/MIMICIV/physionet.org/files/mimiciv/3.1/hosp')
note_path = Path('../MIMIC/MIMICIV/physionet.org/files/mimic-iv-note/2.2/note')
discharge_data_path = note_path / 'discharge.csv.gz'
radiology_data_path = note_path / 'radiology.csv.gz'

In [3]:
def list_files_in_directory(directory, level=0, max_depth=2):
    if level >= max_depth:
        return
    
    for item in sorted(directory.iterdir()):
        indent = '    ' * level
        if item.is_dir():
            if level == max_depth - 1:
                count = sum(1 for _ in item.iterdir())
                print(f"{indent}[DIR] {item.name}: {count} items")
            else:
                print(f"{indent}[DIR] {item.name}")
                list_files_in_directory(item, level + 1, max_depth)
        else:
            print(f"{indent}{item.name}")

if data_path.exists():
    list_files_in_directory(data_path, max_depth=2)
else:
    print(f"Directory '{data_path}' does not exist.")

[DIR] .ipynb_checkpoints
    mimic-cxr-2.1.0-test-set-labeled-checkpoint.csv
IMAGE_FILENAMES
LICENSE.txt
README
SHA256SUMS.txt
[DIR] files
    [DIR] p10: 6396 items
    [DIR] p11: 6571 items
    [DIR] p12: 6526 items
    [DIR] p13: 6548 items
    [DIR] p14: 6506 items
    [DIR] p15: 6592 items
    [DIR] p16: 6476 items
    [DIR] p17: 6642 items
    [DIR] p18: 6543 items
    [DIR] p19: 6579 items
mimic-cxr-2.0.0-chexpert.csv.gz
mimic-cxr-2.0.0-metadata.csv.gz
mimic-cxr-2.0.0-negbio.csv.gz
mimic-cxr-2.0.0-split.csv.gz
mimic-cxr-2.1.0-test-set-labeled.csv


In [4]:
# Get all CSV and CSV.GZ files in the directory
metadata_path = Path('../MIMIC/physionet.org/content/mimic-cxr-jpg/get-zip/2.1.0/mimic-cxr-jpg-chest-radiographs-with-structured-labels-2.1.0/mimic-cxr-2.0.0-metadata.csv.gz')

df = pd.read_csv(metadata_path, compression="gzip" if metadata_path.suffix == ".gz" else None)
    
# Print DataFrame basic information
print("DataFrame Info:")
print(df.info())

# Print the first 5 rows
print("\nFirst 5 Rows:")
print(df.head())

# Print column names
print("\nColumn Names:")
print(df.columns.tolist())

# Print data types and non-null values count
print("\nData Types and Non-Null Value Count:")
print(df.dtypes)

# Print descriptive statistics for numeric columns
print("\nNumeric Column Statistics:")
print(df.describe())

# Print unique value counts for each column
print("\nUnique Value Count per Column:")
print(df.nunique())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377110 entries, 0 to 377109
Data columns (total 12 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   dicom_id                                    377110 non-null  object 
 1   subject_id                                  377110 non-null  int64  
 2   study_id                                    377110 non-null  int64  
 3   PerformedProcedureStepDescription           341598 non-null  object 
 4   ViewPosition                                361341 non-null  object 
 5   Rows                                        377110 non-null  int64  
 6   Columns                                     377110 non-null  int64  
 7   StudyDate                                   377110 non-null  int64  
 8   StudyTime                                   377110 non-null  float64
 9   ProcedureCodeSequence_CodeMeaning           377110 non

In [5]:
# Get all CSV and CSV.GZ files in the directory
df_icustay = pd.read_csv(icustay_path, compression="gzip" if metadata_path.suffix == ".gz" else None)
    
# Print DataFrame basic information
print("DataFrame Info:")
print(df.info())

# Print the first 5 rows
print("\nFirst 5 Rows:")
print(df.head())

# Print column names
print("\nColumn Names:")
print(df.columns.tolist())

# Print data types and non-null values count
print("\nData Types and Non-Null Value Count:")
print(df.dtypes)

# Print descriptive statistics for numeric columns
print("\nNumeric Column Statistics:")
print(df.describe())

# Print unique value counts for each column
print("\nUnique Value Count per Column:")
print(df.nunique())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377110 entries, 0 to 377109
Data columns (total 12 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   dicom_id                                    377110 non-null  object 
 1   subject_id                                  377110 non-null  int64  
 2   study_id                                    377110 non-null  int64  
 3   PerformedProcedureStepDescription           341598 non-null  object 
 4   ViewPosition                                361341 non-null  object 
 5   Rows                                        377110 non-null  int64  
 6   Columns                                     377110 non-null  int64  
 7   StudyDate                                   377110 non-null  int64  
 8   StudyTime                                   377110 non-null  float64
 9   ProcedureCodeSequence_CodeMeaning           377110 non

In [6]:
df["StudyDateStr"] = df["StudyDate"].astype(str)
def format_time(t):
    try:
        t = float(t)
        int_part = int(t)
        decimal_part = f"{t:.6f}".split(".")[1]  
        int_str = str(int_part).zfill(6)       
        return f"{int_str}.{decimal_part}"
    except:
        return "000000.000000"
df["StudyDateStr"] = df["StudyDate"].astype(str).str.zfill(8)
df["StudyTimeStr"] = df["StudyTime"].apply(format_time)

df["StudyDateTime"] = pd.to_datetime(
    df["StudyDateStr"] + " " + df["StudyTimeStr"],
    format="%Y%m%d %H%M%S.%f",
    errors="coerce"
)
print(df[["StudyDate", "StudyTime", "StudyDateTime"]])

        StudyDate   StudyTime           StudyDateTime
0        21800506  213014.531 2180-05-06 21:30:14.531
1        21800506  213014.531 2180-05-06 21:30:14.531
2        21800626  165500.312 2180-06-26 16:55:00.312
3        21800626  165500.312 2180-06-26 16:55:00.312
4        21800723   80556.875 2180-07-23 08:05:56.875
...           ...         ...                     ...
377105   21520708  224550.171 2152-07-08 22:45:50.171
377106   21520708  224550.171 2152-07-08 22:45:50.171
377107   21451104   51448.218 2145-11-04 05:14:48.218
377108   21451102  202809.234 2145-11-02 20:28:09.234
377109   21451103   50507.625 2145-11-03 05:05:07.625

[377110 rows x 3 columns]


In [7]:
def report_nat_rows(df, datetime_col="StudyDateTime", preview=5):
    if datetime_col not in df.columns:
        raise ValueError(f"❌ '{datetime_col}' is not in DataFrame.")

    nat_rows = df[df[datetime_col].isna()]
    count = len(nat_rows)
    print(f"⚠️ '{datetime_col}' Nat counts: {count}")

    if count > 0:
        print(f"\n📋 First {preview} as：")
        display(nat_rows.head(preview))
    
    return nat_rows
bad_rows = report_nat_rows(df, datetime_col="StudyDateTime")

⚠️ 'StudyDateTime' Nat counts: 0


In [8]:
# df["StudyDate"] = pd.to_datetime(df["StudyDate"].astype(str), format="%Y%m%d", errors="coerce")
df_sorted = df.sort_values(by=["subject_id", "StudyDateTime","ViewPosition"])
df_sorted["order"] = df_sorted.groupby("subject_id").cumcount() + 1
# df_sorted = df_sorted[df_sorted["ViewPosition"] == "AP"]
df_final = df_sorted[["subject_id", "study_id", "StudyDateTime" ,"dicom_id", "ViewPosition" ,"order"]]

In [9]:
df_final.head(10)

,subject_id,study_id,StudyDateTime,dicom_id,ViewPosition,order
1,10000032,50414267,2180-05-06 21:30:14.531,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,LATERAL,1
0,10000032,50414267,2180-05-06 21:30:14.531,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,PA,2
3,10000032,53189527,2180-06-26 16:55:00.312,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,LATERAL,3
2,10000032,53189527,2180-06-26 16:55:00.312,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,PA,4
4,10000032,53911762,2180-07-23 08:05:56.875,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,AP,5
5,10000032,53911762,2180-07-23 08:05:56.875,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,AP,6
6,10000032,56699142,2180-08-05 23:44:24.765,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,AP,7
7,10000764,57375967,2132-10-15 08:40:47.984,096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4,AP,1
8,10000764,57375967,2132-10-15 08:40:47.984,b79e55c3-735ce5ac-64412506-cdc9ea79-f1af521f,LATERAL,2
9,10000764,57375967,2132-10-15 08:40:47.984,dcfeeac4-1597e318-d0e6736a-8b2c2238-47ac3f1b,LATERAL,3


In [10]:
def report_time_spans(df_sorted, threshold_hours=24):
    df_sorted = df_sorted.copy()

    if "StudyDateTime" not in df_sorted.columns:
        raise ValueError("❗️Missing 'StudyDateTime' column. Please create it before running this function.")

    df_sorted = df_sorted[df_sorted["StudyDateTime"].notna()] 
    span_info = []

    for subject_id, group in df_sorted.groupby("subject_id"):
        group = group.sort_values("StudyDateTime").reset_index(drop=True)
        t0 = group["StudyDateTime"].iloc[0]
        t1 = group["StudyDateTime"].iloc[-1]
        span_hours = (t1 - t0).total_seconds() / 3600 if len(group) > 1 else 0.0

        span_info.append({
            "subject_id": subject_id,
            "start_time": t0,
            "end_time": t1,
            "span_hours": span_hours
        })

    span_df = pd.DataFrame(span_info)
    total = len(span_df)
    below_thresh = (span_df["span_hours"] < threshold_hours).sum()
    print(f"✅ {total} subject_ids in total; among them, {below_thresh} have a time span shorter than {threshold_hours} hours ({below_thresh/total:.1%}).")

    return span_df

def filter_by_time_window(df, first_24=False, last_24=False, last_48=False, return_df=False):
    df = df.copy()
    
    if "StudyDateTime" not in df.columns:
        raise ValueError("❌ No 'StudyDateTime'")

    df = df[df["StudyDateTime"].notna()] 
    result_frames = []

    for subject_id, group in df.groupby("subject_id"):
        group = group.sort_values("StudyDateTime").reset_index(drop=True)

        if len(group) == 1:
            result_frames.append(group)
            continue

        t0 = group["StudyDateTime"].iloc[0]
        t_end = group["StudyDateTime"].iloc[-1]

        group["relative_hours_from_start"] = (group["StudyDateTime"] - t0).dt.total_seconds() / 3600
        group["relative_hours_to_end"] = (t_end - group["StudyDateTime"]).dt.total_seconds() / 3600

        selected_rows = pd.DataFrame()

        if first_24:
            selected_rows = pd.concat([selected_rows, group[group["relative_hours_from_start"] <= 24]])

        if last_24:
            selected_rows = pd.concat([selected_rows, group[group["relative_hours_to_end"] <= 24]])

        if last_48:
            selected_rows = pd.concat([selected_rows, group[group["relative_hours_to_end"] <= 48]])

        if not selected_rows.empty:
            result_frames.append(selected_rows)

    if result_frames:
        combined = pd.concat(result_frames).drop_duplicates()
        total = combined["dicom_id"].nunique()
        print(f"✅ Total selected dicom_id: {total}")
        if return_df:
            return combined
    else:
        print("⚠️ No time window selected or no StudyDateTime available.")
        if return_df:
            return pd.DataFrame()

In [11]:
span_df = report_time_spans(df_final, threshold_hours=24)

✅ 65379 subject_ids in total; among them, 34106 have a time span shorter than 24 hours (52.2%).


In [12]:
df_merged = df_final.merge(df_icustay[["subject_id", "stay_id", "intime" ,"outtime","los"]], on='subject_id', how='left')

In [13]:
# df_merged[df_merged['subject_id'].isin([10000032, 10001217])]

In [14]:
# df_merged[df_merged['stay_id'].notna()]

In [15]:
df_merged['StudyDateTime'] = pd.to_datetime(df_merged['StudyDateTime'])
df_merged['intime'] = pd.to_datetime(df_merged['intime'])
df_merged['outtime'] = pd.to_datetime(df_merged['outtime'])

In [16]:
no_icu_count = df_merged['stay_id'].isna().sum()
icu_counts = df_merged.dropna(subset=['stay_id']).groupby('subject_id')['stay_id'].nunique()
more_than_once_icu = (icu_counts > 1).sum()
df_has_icu = df_merged[df_merged['stay_id'].notna()]
before_icu = df_has_icu[df_has_icu['StudyDateTime'] < df_has_icu['intime']].shape[0]
within_icu = df_has_icu[(df_has_icu['StudyDateTime'] >= df_has_icu['intime']) & 
                        (df_has_icu['StudyDateTime'] <= df_has_icu['outtime'])].shape[0]
after_icu = df_has_icu[df_has_icu['StudyDateTime'] > df_has_icu['outtime']].shape[0]

In [17]:
print("Summary:")
print(f"1. Number of images without an ICU stay: {no_icu_count}")
print(f"2. Number of subject_ids with more than one ICU stay: {more_than_once_icu}")
print(f"3. Distribution of StudyDateTime relative to ICU stay:")
print(f"   - Before ICU stay: {before_icu}")
print(f"   - During ICU stay: {within_icu}")
print(f"   - After ICU stay: {after_icu}")


Summary:
1. Number of images without an ICU stay: 178089
2. Number of subject_ids with more than one ICU stay: 7186
3. Distribution of StudyDateTime relative to ICU stay:
   - Before ICU stay: 257099
   - During ICU stay: 59415
   - After ICU stay: 210609


In [18]:
first_icu = (
    df_merged[df_merged['stay_id'].notna()]
    .sort_values(['subject_id', 'intime']) 
    .drop_duplicates(subset='subject_id', keep='first') 
)
df_first_icu = df_merged[['subject_id', 'StudyDateTime', 'dicom_id', 'ViewPosition', 'order']].drop_duplicates()
df_first_icu = df_first_icu.merge(
    first_icu[['subject_id', 'stay_id', 'intime', 'outtime', 'los']], 
    on='subject_id', how='left'
)
df_first_icu['StudyDateTime'] = pd.to_datetime(df_first_icu['StudyDateTime'])
df_first_icu['intime'] = pd.to_datetime(df_first_icu['intime'])
df_first_icu['outtime'] = pd.to_datetime(df_first_icu['outtime'])

no_icu_count = df_first_icu['stay_id'].isna().sum()

before_icu = df_first_icu[df_first_icu['StudyDateTime'] < df_first_icu['intime']].shape[0]
within_icu = df_first_icu[(df_first_icu['StudyDateTime'] >= df_first_icu['intime']) &
                          (df_first_icu['StudyDateTime'] <= df_first_icu['outtime'])].shape[0]
after_icu = df_first_icu[df_first_icu['StudyDateTime'] > df_first_icu['outtime']].shape[0]

print("Summary:")
print(f"1. Number of images without an ICU stay: {no_icu_count}")
print(f"2. Distribution of StudyDateTime relative to the first ICU stay:")
print(f"   - Before first ICU stay: {before_icu}")
print(f"   - During first ICU stay: {within_icu}")
print(f"   - After first ICU stay: {after_icu}")

Summary:
1. Number of images without an ICU stay: 178089
2. Distribution of StudyDateTime relative to the first ICU stay:
   - Before first ICU stay: 55906
   - During first ICU stay: 37816
   - After first ICU stay: 105299


In [19]:
before_icu_ids = df_first_icu[df_first_icu['StudyDateTime'] < df_first_icu['intime']]['subject_id'].unique()
within_icu_ids = df_first_icu[
    (df_first_icu['StudyDateTime'] >= df_first_icu['intime']) &
    (df_first_icu['StudyDateTime'] <= df_first_icu['outtime'])
]['subject_id'].unique()
after_icu_ids = df_first_icu[df_first_icu['StudyDateTime'] > df_first_icu['outtime']]['subject_id'].unique()
total_unique_subjects = df_first_icu['subject_id'].nunique()
no_icu_ids = df_first_icu[df_first_icu['stay_id'].isna()]['subject_id'].unique()
num_no_icu = len(no_icu_ids)
print("Summary:")
print(f"1. Total unique subject_ids: {total_unique_subjects}")
print(f"2. Number of subject_ids with more than one ICU stay: {more_than_once_icu}")
print(f"3. Unique subject_id counts by StudyDateTime (relative to first ICU stay):")
print(f"   - Before first ICU stay: {len(before_icu_ids)}")
print(f"   - During first ICU stay: {len(within_icu_ids)}")
print(f"   - After first ICU stay: {len(after_icu_ids)}")
print(f"   - No ICU stay: {len(no_icu_ids)}")

Summary:
1. Total unique subject_ids: 65379
2. Number of subject_ids with more than one ICU stay: 7186
3. Unique subject_id counts by StudyDateTime (relative to first ICU stay):
   - Before first ICU stay: 12739
   - During first ICU stay: 9333
   - After first ICU stay: 11910
   - No ICU stay: 45277


In [20]:
has_icu_ids = df_first_icu[df_first_icu['stay_id'].notna()]['subject_id'].unique()

true_no_icu_ids = set(no_icu_ids) - set(has_icu_ids)
num_true_no_icu = len(true_no_icu_ids)

print("Summary (exclude duplicates):")
print(f"1. Total unique subject_ids: {total_unique_subjects}")
print(f"2. Number of subject_ids with more than one ICU stay: {more_than_once_icu}")
print(f"3. Unique subject_id counts by StudyDateTime (relative to first ICU stay):")
print(f"   - Before first ICU stay: {len(before_icu_ids)}")
print(f"   - During first ICU stay: {len(within_icu_ids)}")
print(f"   - After first ICU stay: {len(after_icu_ids)}")
print(f"   - No ICU stay (never appeared in ICU at all): {num_true_no_icu}")

Summary (exclude duplicates):
1. Total unique subject_ids: 65379
2. Number of subject_ids with more than one ICU stay: 7186
3. Unique subject_id counts by StudyDateTime (relative to first ICU stay):
   - Before first ICU stay: 12739
   - During first ICU stay: 9333
   - After first ICU stay: 11910
   - No ICU stay (never appeared in ICU at all): 45277


In [21]:
df_discharge = pd.read_csv(discharge_data_path)
df_radiology = pd.read_csv(radiology_data_path)

df_discharge = df_discharge[df_discharge['text'].notna()]
df_radiology = df_radiology[df_radiology['text'].notna()]

discharge_ids = set(df_discharge['subject_id'].unique())
radiology_ids = set(df_radiology['subject_id'].unique())

all_note_subjects = discharge_ids.union(radiology_ids)

print(f"Discharge note subject count: {len(discharge_ids)}")
print(f"Radiology note subject count: {len(radiology_ids)}")
print(f"Total unique subjects with notes: {len(all_note_subjects)}")

Discharge note subject count: 145914
Radiology note subject count: 237427
Total unique subjects with notes: 245952


In [22]:
# df_discharge

In [23]:
# df_radiology

In [24]:
df_discharge['note_type'] = 'discharge'
df_radiology['note_type'] = 'radiology'

df_note = pd.concat([df_discharge, df_radiology], ignore_index=True)

df_note['charttime'] = pd.to_datetime(df_note['charttime'])

df_note = df_note[df_note['text'].notna()]

print(df_note.head())

          note_id  subject_id     hadm_id  note_type  note_seq  charttime  \
0  10000032-DS-21    10000032  22595853.0  discharge        21 2180-05-07   
1  10000032-DS-22    10000032  22841357.0  discharge        22 2180-06-27   
2  10000032-DS-23    10000032  29079034.0  discharge        23 2180-07-25   
3  10000032-DS-24    10000032  25742920.0  discharge        24 2180-08-07   
4  10000084-DS-17    10000084  23052089.0  discharge        17 2160-11-25   

             storetime                                               text  
0  2180-05-09 15:26:00   \nName:  ___                     Unit No:   _...  
1  2180-07-01 10:15:00   \nName:  ___                     Unit No:   _...  
2  2180-07-25 21:42:00   \nName:  ___                     Unit No:   _...  
3  2180-08-10 05:43:00   \nName:  ___                     Unit No:   _...  
4  2160-11-25 15:09:00   \nName:  ___                    Unit No:   __...  


In [25]:
note_ids = set(df_note['subject_id'].unique())

before_with_notes = set(before_icu_ids) & note_ids
within_with_notes = set(within_icu_ids) & note_ids
after_with_notes = set(after_icu_ids) & note_ids
true_no_icu_with_notes = true_no_icu_ids & note_ids 

print("Summary:")
print(f"1. Total unique subject_ids: {total_unique_subjects}")
# print(f"2. Number of subject_ids with more than one ICU stay: {more_than_once_icu}")
print(f"2. Before first ICU stay with notes: {len(before_with_notes)}")
print(f"3. During first ICU stay with notes: {len(within_with_notes)}")
print(f"4. After first ICU stay with notes: {len(after_with_notes)}")
print(f"5. No ICU stay at all with notes: {len(true_no_icu_with_notes)}")

Summary:
1. Total unique subject_ids: 65379
2. Before first ICU stay with notes: 12738
3. During first ICU stay with notes: 9333
4. After first ICU stay with notes: 11910
5. No ICU stay at all with notes: 41640


In [36]:
subjects_with_notes = set(df_note['subject_id'].unique())

df_first_icu['StudyDateTime'] = pd.to_datetime(df_first_icu['StudyDateTime'])
df_first_icu['intime'] = pd.to_datetime(df_first_icu['intime'])

images_within_24h = df_first_icu[
    df_first_icu['StudyDateTime'] <= df_first_icu['intime'] + pd.Timedelta(hours=24)
]

images_within_48h = df_first_icu[
    df_first_icu['StudyDateTime'] <= df_first_icu['intime'] + pd.Timedelta(hours=48)
]

subjects_with_images_24h = set(images_within_24h['subject_id'].unique())
subjects_with_images_48h = set(images_within_48h['subject_id'].unique())

cohort_subjects_24h = subjects_with_images_24h & subjects_with_notes
cohort_subjects_48h = subjects_with_images_48h & subjects_with_notes

print("In-ICU mortality cohort (based on image before/within first ICU stay + clinical notes):")
print(f"1. Subjects with CXR before or within 24h and clinical note: {len(cohort_subjects_24h)}")
print(f"2. Subjects with CXR before or within 48h and clinical note: {len(cohort_subjects_48h)}")

In-ICU mortality cohort (based on image before/within first ICU stay + clinical notes):
1. Subjects with CXR before or within 24h and clinical note: 15513
2. Subjects with CXR before or within 48h and clinical note: 16186


In [40]:
images_within_visit = df_first_icu[
    df_first_icu['StudyDateTime'] <= df_first_icu['outtime']
]
subjects_with_images = set(images_within_visit['subject_id'].unique())
cohort_subjects_all = subjects_with_images & subjects_with_notes
print(f"3. Subjects with CXR before or during and clinical note: {len(cohort_subjects)}")

3. Subjects with CXR before or during and clinical note: 16206


In [38]:
df_first_icu['StudyDateTime'] = pd.to_datetime(df_first_icu['StudyDateTime'])
df_first_icu['outtime'] = pd.to_datetime(df_first_icu['outtime'])

images_within_last_48h = df_first_icu[
    (df_first_icu['StudyDateTime'] <= df_first_icu['outtime']) &
    (df_first_icu['StudyDateTime'] >= df_first_icu['outtime'] - pd.Timedelta(hours=48))
]

subjects_with_images_last_48h = set(images_within_last_48h['subject_id'].unique())

cohort_subjects_last_48h = subjects_with_images_last_48h & subjects_with_notes

print("Readmission cohort (based on image within last 48h of first ICU stay + clinical notes):")
print(f"Subjects with CXR in last 48h before ICU discharge and clinical note: {len(cohort_subjects_last_48h)}")

Readmission cohort (based on image within last 48h of first ICU stay + clinical notes):
Subjects with CXR in last 48h before ICU discharge and clinical note: 9700


In [41]:
def extract_cxr_metadata_by_subjects(
    metadata_path: str,
    cohort_24h: set,
    cohort_48h: set,
    cohort_l48h: set,
    cohort_all: set,
    output_24h_path: str = "cxr_f24h_metadata.csv.gz",
    output_48h_path: str = "cxr_f48h_metadata.csv.gz",
    output_l48h_path: str = "cxr_l48h_metadata.csv.gz",
    output_all_path: str = "cxr_all_metadata.csv.gz"
):

    df_metadata = pd.read_csv(metadata_path)

    df_24h = df_metadata[df_metadata['subject_id'].isin(cohort_24h)]
    df_48h = df_metadata[df_metadata['subject_id'].isin(cohort_48h)]
    df_l48h = df_metadata[df_metadata['subject_id'].isin(cohort_l48h)]
    df_all = df_metadata[df_metadata['subject_id'].isin(cohort_all)]

    df_24h.to_csv(output_24h_path, index=False, compression='gzip')
    df_48h.to_csv(output_48h_path, index=False, compression='gzip')
    df_l48h.to_csv(output_l48h_path, index=False, compression='gzip')
    df_all.to_csv(output_all_path, index=False, compression='gzip')

    print(f"✅ Saved {len(df_24h)} rows to {output_24h_path}")
    print(f"✅ Saved {len(df_48h)} rows to {output_48h_path}")
    print(f"✅ Saved {len(df_l48h)} rows to {output_l48h_path}")
    print(f"✅ Saved {len(df_all)} rows to {output_all_path}")


In [42]:
# extract_cxr_metadata_by_subjects(
#     metadata_path=metadata_path,
#     cohort_24h=cohort_subjects_24h,
#     cohort_48h=cohort_subjects_48h,
#     cohort_l48h=cohort_subjects_last_48h,
#     cohort_all=cohort_subjects_all,
# )

✅ Saved 155417 rows to cxr_f24h_metadata.csv.gz
✅ Saved 160345 rows to cxr_f48h_metadata.csv.gz
✅ Saved 104339 rows to cxr_l48h_metadata.csv.gz
✅ Saved 160928 rows to cxr_all_metadata.csv.gz


In [46]:
def extract_cxr_metadata_by_time(
    metadata_path: str,
    df_first_icu: pd.DataFrame,
    output_24h_path: str = "cxr_f24h_metadata.csv.gz",
    output_48h_path: str = "cxr_f48h_metadata.csv.gz",
    output_l48h_path: str = "cxr_l48h_metadata.csv.gz",
    output_all_path: str = "cxr_all_metadata.csv.gz"
):
    df_metadata = pd.read_csv(metadata_path)
    original_columns = df_metadata.columns.tolist() 

    if 'StudyDateTime' not in df_metadata.columns:
        if 'StudyDate' in df_metadata.columns and 'StudyTime' in df_metadata.columns:
            df_metadata['StudyDateStr'] = df_metadata['StudyDate'].astype(str).str.zfill(8)

            def format_time(t):
                try:
                    t = float(t)
                    int_part = int(t)
                    decimal_part = f"{t:.6f}".split(".")[1]
                    int_str = str(int_part).zfill(6)
                    return f"{int_str}.{decimal_part}"
                except:
                    return "000000.000000"

            df_metadata['StudyTimeStr'] = df_metadata['StudyTime'].apply(format_time)
            df_metadata['StudyDateTime'] = pd.to_datetime(
                df_metadata['StudyDateStr'] + " " + df_metadata['StudyTimeStr'],
                format="%Y%m%d %H%M%S.%f",
                errors='coerce'
            )
        else:
            raise ValueError("Metadata missing both 'StudyDateTime' and ('StudyDate', 'StudyTime') to generate StudyDateTime.")

    df_first_icu['intime'] = pd.to_datetime(df_first_icu['intime'])
    df_first_icu['outtime'] = pd.to_datetime(df_first_icu['outtime'])

    df = df_metadata.merge(df_first_icu[['subject_id', 'intime', 'outtime']], on='subject_id', how='left')

    df_24h = df[
        df['StudyDateTime'] <= df['intime'] + pd.Timedelta(hours=24)
    ]
    df_48h = df[
        df['StudyDateTime'] <= df['intime'] + pd.Timedelta(hours=48)
    ]
    df_l48h = df[
        (df['StudyDateTime'] <= df['outtime']) &
        (df['StudyDateTime'] >= df['outtime'] - pd.Timedelta(hours=48))
    ]
    df_all = df[
        (df['StudyDateTime'] <= df['outtime']) 
    ]

    df_24h[original_columns].to_csv(output_24h_path, index=False, compression='gzip')
    df_48h[original_columns].to_csv(output_48h_path, index=False, compression='gzip')
    df_l48h[original_columns].to_csv(output_l48h_path, index=False, compression='gzip')
    df_all[original_columns].to_csv(output_all_path, index=False, compression='gzip')

    print(f"✅ Saved {len(df_24h)} rows to {output_24h_path}")
    print(f"✅ Saved {len(df_48h)} rows to {output_48h_path}")
    print(f"✅ Saved {len(df_l48h)} rows to {output_l48h_path}")
    print(f"✅ Saved {len(df_all)} rows to {output_all_path}")

In [48]:
extract_cxr_metadata_by_time(
    metadata_path=metadata_path,
    df_first_icu=df_first_icu,
    output_24h_path="cxr_filtered_f24h.csv.gz",
    output_48h_path="cxr_filtered_f48h.csv.gz",
    output_l48h_path="cxr_filtered_l48h.csv.gz",
    output_all_path="cxr_filtered_all.csv.gz"
)

✅ Saved 1148785 rows to cxr_filtered_f24h.csv.gz
✅ Saved 1262373 rows to cxr_filtered_f48h.csv.gz
✅ Saved 211872 rows to cxr_filtered_l48h.csv.gz
✅ Saved 1774333 rows to cxr_filtered_all.csv.gz


In [ ]:
# drop duplicate